In [1]:
import numpy as np
import plotly.graph_objects as go

# from multiprocess import Pool
import os
import gzip
import pickle as pkl

import numpy as np
import plotly.graph_objects as go

import games


import os
import gzip
import pickle as pkl

def loadall_results(path, horizon, n_folds):
    result = np.zeros( (n_folds, 19999) )
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):
            try:
                result[i] = pkl.load(f)[:19999]
            except EOFError:
                break
    return result


In [ ]:
# def loadall_results(path, horizon, n_folds):
#     result = np.zeros( (n_folds, horizon) )
#     with gzip.open(  path ,'rb') as f:
#         for i in range(n_folds):
#             try:
#                 result[i] = pkl.load(f)
#             except EOFError:
#                 break
#     return result

# name = 'AT'
# task = 'easy'
# context_type = 'quintic'


# n_folds = 96
# horizon = 100000

# colors = [  [0,0,0],  [250,0,0], [0,250,0]  ] 
# labels = [  'random', 'CBPside005',  'RandCBPside005' ]
# labels_latex = [  'random', 'CBPside',  'RandCBPside'  ] 

# final_regrets = {}

# for label, label_latex in zip(labels, labels_latex):

#     direct = './contextual_results/{}'.format(name)
#     path = os.path.join(direct, '{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )
#     result = loadall_results(path, horizon, n_folds)
#     final_regrets[label_latex] =  result[:,-1] 

# final = np.array( [ (1, i) if final_regrets['RandCBPside'][i] <= final_regrets['CBPside'][i] else (0, i) for i in range(n_folds) ] )

# print( sum(final[:,0]) )



In [ ]:





name = 'LE'
task = 'balanced'
context_type = 'linear'
epsilon = 10e-7 #0.1

n_folds = 96
horizon = 20000
d = 2


colors = [  [250,0,0], [250,0,0], [250,0,0], [250,0,0], [250,0,0],
            [0,250,0], [0,250,0], [0,250,0], [0,250,0], [0,250,0],
            [0,0,250], [0,0,250], [0,0,250], [0,0,250],[0,0,250],
            [255,128,0], [255,128,0], [255,128,0], [255,128,0], [255,128,0]  ]

labels_latex = [    'Gaussian_1_5_10e7',  'Gaussian_2_5_10e7', 'Gaussian_10_5_10e7', 
                    'Gaussian_1_10_10e7',  'Gaussian_2_10_10e7', 'Gaussian_10_10_10e7',
                   'Gaussian_1_20_10e7',  'Gaussian_2_20_10e7', 'Gaussian_10_20_10e7',
                    ]

labels = [      'RandCBPside2_1_5_07', 'RandCBPside2_2_5_07', 'RandCBPside2_10_5_07',
                 'RandCBPside2_1_10_07', 'RandCBPside2_2_10_07', 'RandCBPside2_10_10_07', 
                'RandCBPside2_1_20_07', 'RandCBPside2_2_20_07', 'RandCBPside2_10_20_07', 
                 ]

fig = go.Figure( )


for color, label, label_latex in zip( colors, labels, labels_latex):
    r,g,b = color
    direct = './contextual_results/{}'.format(name)
    path = os.path.join(direct, 'cr_benchmark_{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )

    result = loadall_results(path, horizon, n_folds)

    worst_case = np.argmax( result[:,-1] )
    # print(worst_case)
    
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    
    # upper_regret = result[worst_case] 

    std =  np.std(result,0) 
    upper_regret = regret + std

    parts = label_latex.rsplit('_', 2)
    first_part = parts[0]
    if 'Gaussian_132' == first_part:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/32'  ) ) 

    elif 'Gaussian_116' == first_part:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdashdot' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1/16'   ) )  

    elif 'Gaussian_18' == first_part:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'dash' ), mode='lines',
                legendgroup='M = {}'.format( label.split('_')[2] ),  name='σ = 1/8'   ) ) 

    elif 'Gaussian_1' == first_part:
        fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ), name='σ = 1'  ) ) 

    elif 'Gaussian_10' == first_part:
        fig.add_trace( go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'longdash' ), mode='lines', 
                legendgroup='M = {}'.format( label.split('_')[2] ),legendgrouptitle_text='M = {}'.format( label.split('_')[2] ), name='σ = 10'  ) ) 
        print(label_latex, regret[-1],std[-1])

    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )

fig.update_layout(  legend= dict(yanchor="top",y=0.97,xanchor="left",x=0.03), #x = 0.25, y=0.15 for center, x= 0.97, y=0.03
                        legend_orientation="h",
                        autosize=False,
                        xaxis_title="Sequence",
                        yaxis_title="Mean Cum. Regret + Standard. Dev.",
                        margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),  )
fig.show()


# fig.update_yaxes(type="log")
# fig.show(legend=True)

# fig.write_image("./benchmarkRandCBPside_{}_{}_{}.pdf".format(task, name, context_type)  )


In [6]:
fig.write_image("./CR_benchmark_{}_{}_{}.pdf".format(task, name, context_type)  )

In [14]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind,mannwhitneyu

dfs = []
n_folds = 96
horizon = 20000
context_type = 'linear'

for game in ['AT', 'LE']:

    for task in ['balanced']:

        labels = [      'RandCBPside2_10_5_07', 'RandCBPside2_1_5_07', 'RandCBPside2_18_5_07', 'RandCBPside2_116_5_07', 'RandCBPside2_132_5_07' ,
                        'RandCBPside2_10_10_07', 'RandCBPside2_1_10_07', 'RandCBPside2_18_10_07', 'RandCBPside2_116_10_07', 'RandCBPside2_132_10_07',
                        'RandCBPside2_10_20_07', 'RandCBPside2_1_20_07', 'RandCBPside2_18_20_07', 'RandCBPside2_116_20_07', 'RandCBPside2_132_20_07',
                        'RandCBPside2_10_100_07', 'RandCBPside2_1_100_07', 'RandCBPside2_18_100_07', 'RandCBPside2_116_100_07', 'RandCBPside2_132_100_07'  ]


        perf_dic = {}

        for label in  labels:

            direct = './contextual_results/{}'.format(game)
            path = os.path.join(direct, 'cr_benchmark_{}_{}_{}_{}_{}.pkl.gz'.format(task,context_type, horizon, n_folds, label) )
            result = loadall_results(path, horizon, n_folds)

            perf_dic[label] = result[:,-1] 

        df = pd.DataFrame.from_dict(perf_dic)

        ############### create data frame:

        # Calculate mean and std
        mean_values = df.mean()
        std_values = df.std()

        # Calculate p-values
        p_values = {}
        for strategy in df.columns:
            if strategy == 'RandCBPside2_10_5_07':
                p =  1
                p_values[strategy] = p
            else:
                s, p = ttest_ind(df['RandCBPside2_10_5_07'], df[strategy], equal_var=False)
                # s, p = mannwhitneyu(df['RandCBP'], df[strategy],  alternative='less')
                # s, p = ttest_ind( df[strategy], df['RandCBP'])
                p_values[strategy] = p

    
        # Find the minimum value within each row
        min_values = df.min(axis=1)
        # Create a boolean DataFrame indicating if each strategy achieved the minimum value
        strategy_won = df.eq(min_values, axis=0)
        # Count how many times each strategy won
        winning_counts = strategy_won.sum()
        # winning_counts = df.apply(idxmin_excluding_duplicates, axis=1).value_counts()

        # Create the new DataFrame
        new_data = {
            'Mean_{}_{}'.format(game,task): mean_values,
            'Std_{}_{}'.format(game,task): std_values,
            'PValue_{}_{}'.format(game,task): p_values,
            'WinningCount_{}_{}'.format(game,task): winning_counts,  }

        pd.options.display.float_format = '{:.3f}'.format
        
        new_df = pd.DataFrame(new_data)
        new_df['WinningCount_{}_{}'.format(game,task)].fillna(0, inplace=True)
        new_df['WinningCount_{}_{}'.format(game,task)] = new_df['WinningCount_{}_{}'.format(game,task)].astype(int) 
        dfs.append(new_df)

In [15]:
dfs

[                         Mean_AT_balanced  Std_AT_balanced  \
 RandCBPside2_10_5_07              648.938           80.020   
 RandCBPside2_1_5_07              1016.312           82.151   
 RandCBPside2_18_5_07             3110.385           87.879   
 RandCBPside2_116_5_07            3213.583           87.408   
 RandCBPside2_132_5_07            3239.635           88.184   
 RandCBPside2_10_10_07             648.938           80.020   
 RandCBPside2_1_10_07             1328.417           84.333   
 RandCBPside2_18_10_07            3478.896           91.884   
 RandCBPside2_116_10_07           3601.583           91.623   
 RandCBPside2_132_10_07           3632.760           91.713   
 RandCBPside2_10_20_07             650.344           79.621   
 RandCBPside2_1_20_07             1453.750           86.290   
 RandCBPside2_18_20_07            3629.281           92.125   
 RandCBPside2_116_20_07           3757.094           92.631   
 RandCBPside2_132_20_07           3790.656           93

In [23]:
df = pd.concat(dfs, axis=1)
# df = df.round(2)

# Sample DataFrame columns
columns = [
    'Mean_AT_balanced', 'Std_AT_balanced', 'Winning Count_AT_balanced', 'P-Value_AT_balanced',
    'Mean_LE_balanced', 'Std_LE_balanced', 'Winning Count_LE_balanced', 'P-Value_LE_balanced'
]

# # Create arrays for each level of the multi-index
first_level = ['mean', 'std', 'pvalue', 'win count',] * 2
second_level = ['balanced', 'balanced', 'balanced', 'balanced'] * 2
third_level = ['AT', 'AT', 'AT',  'AT',  'LE', 'LE', 'LE', 'LE'] 

# # Create a MultiIndex from the arrays
multi_index = pd.MultiIndex.from_arrays([third_level , second_level, first_level], names=('Game', 'Case', 'Metric'))

# # Create a DataFrame with the specified columns and multi-index
# df = pd.DataFrame(columns=columns, index=[0])
# df.columns = multi_index

# # desired_order = [ 'RandCBPside2_10_5_07', 'RandCBPside2_10_10_07', 'RandCBPside2_10_20_07', 'RandCBPside2_10_100_07'   ]
# # df = df.reindex(desired_order)

# df = df.round(3)

df


,Mean_AT_balanced,Std_AT_balanced,PValue_AT_balanced,WinningCount_AT_balanced,Mean_LE_balanced,Std_LE_balanced,PValue_LE_balanced,WinningCount_LE_balanced
RandCBPside2_10_5_07,648.938,80.020,1.000,86,1173.281,86.249,1.000,79
RandCBPside2_1_5_07,1016.312,82.151,0.000,0,2026.604,70.161,0.000,0
RandCBPside2_18_5_07,3110.385,87.879,0.000,0,5771.542,99.845,0.000,0
RandCBPside2_116_5_07,3213.583,87.408,0.000,0,5938.760,99.893,0.000,0
RandCBPside2_132_5_07,3239.635,88.184,0.000,0,5981.875,99.923,0.000,0
RandCBPside2_10_10_07,648.938,80.020,1.000,86,1173.281,86.249,1.000,79
RandCBPside2_1_10_07,1328.417,84.333,0.000,0,2373.229,92.964,0.000,0
RandCBPside2_18_10_07,3478.896,91.884,0.000,0,6385.260,97.834,0.000,0
RandCBPside2_116_10_07,3601.583,91.623,0.000,0,6597.396,99.217,0.000,0
RandCBPside2_132_10_07,3632.760,91.713,0.000,0,6651.000,99.229,0.000,0


In [24]:
df.to_csv('dataframe_randcbpside.csv')